# Imports and Setup

In [1]:
import pandas as pd
# from pandas.io.json import json_normalize
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import json
from datetime import date, datetime, timedelta
import time
import os
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
import pathlib
#from config import ACCESS_KEY,SECRET_KEY
# import polling
import boto3
from osgeo import gdal
import glob
import sys



# Generating Order
# API Key stored as an env variable
PL_API_KEY = 'fa49aea30c534483bbf74f6ec9bc311a' # Jason
PL_API_KEY = 'd1522b0573f548ea8233d09b9a917b74' # Conor
PLANET_API_KEY = PL_API_KEY #os.getenv('PL_API_KEY')
orders_v2_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}


item_type = "PSScene4Band"

# set up options for conversion to jpg
gdal_Translate_options_list = [
    '-ot Byte',
    '-of JPEG',
    '-b 1',
    '-b 2',
    '-b 3',
    '-b 4',
    '-scale min_val max_val'
] 
gdal_Translate_options_string = " ".join(gdal_Translate_options_list)


# Load waypoints into a GeoDataFrame


In [2]:
# Load waypoints into a GeoDataFrame
# waypoint_data = {
#     'Waypoint' : ['Orangi River', 'Mara River', 'Sand River', 'Grumeti River', 'Lake Masek', 
#                   'Lake Magadi', 'Lake Empakaai', 'Lake Magadi 2', 'Mbalageti River', 'Ruwana River', 'Talek River'],
#     'latitude' : [-2.302313, -1.562928, -1.595733, -2.249034, -3.024818, -3.202214, -2.915433, -2.656248, -2.603015, -2.044819, -1.416096],
#     'longitude' : [34.830777, 34.997068, 35.069241, 34.486842, 35.038474, 35.536431, 35.841355, 34.788239, 34.720511, 34.230374, 35.097661]
# }

waypoint_data = {
    'Waypoint' : ['Orangi River'],
    'latitude' : [-2.302313],
    'longitude' : [34.830777]
}


waypoint_df = pd.DataFrame(waypoint_data)
waypoint_gdf = gpd.GeoDataFrame(waypoint_df, geometry=gpd.points_from_xy(waypoint_df.longitude, waypoint_df.latitude))

# Applying WGS84 to the CRS
waypoint_gdf.crs = {'init' :'epsg:4326'} 

waypoint_gdf

/Users/piper/anaconda2/envs/py36/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Waypoint,latitude,longitude,geometry
0,Orangi River,-2.302313,34.830777,POINT (34.83078 -2.30231)


# Prep GeoDataFrame with buffers


In [3]:
# Converting geodataframe to Meters from Lat/Long
# Allows for square buffer to be applied (450m)
point_gdf_m = waypoint_gdf.to_crs(epsg=3395)
# point_gdf_m

# Applying the buffer, cap_style = 3 --> Square Buffer, 383.5 = 256x256 chip size
buffer = point_gdf_m.buffer(382.30, cap_style=3)

# # Convert buffer back to WGS84 Lat/Long
buffer_wgs84 = buffer.to_crs(epsg=4326)


In [4]:
# Merging GDF and DF to get the Waypoint names
joined_buffer_wgs84 = pd.concat([waypoint_df,buffer_wgs84], axis=1)
joined_buffer_wgs84 = joined_buffer_wgs84.rename(columns = {0:'polygon'}).set_geometry('polygon')

joined_buffer_wgs84_drop = joined_buffer_wgs84.drop(['geometry'], axis=1)
joined_buffer_wgs84_json = joined_buffer_wgs84_drop.to_json()

# transforming to json for inclusion into Planet API
buffer_wgs84_json_parsed = json.loads(joined_buffer_wgs84_json)
buffer_wgs84_json_api = buffer_wgs84_json_parsed['features']#[0]['geometry']['coordinates']

today = datetime.isoformat(datetime.utcnow())+'Z'#(datetime.today())
start_date = datetime.isoformat(datetime.utcnow() - timedelta(7)) + 'Z'

In [5]:
buffer_wgs84_json_api

[{'id': '0',
  'type': 'Feature',
  'properties': {'Waypoint': 'Orangi River',
   'latitude': -2.302313,
   'longitude': 34.830777},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[34.834211259331184, -2.2988584194452644],
     [34.834211259331184, -2.305767572007151],
     [34.827342740668804, -2.305767572007151],
     [34.827342740668804, -2.2988584194452644],
     [34.834211259331184, -2.2988584194452644]]]}}]

In [6]:
# Getting Image ID's for each waypoint that has the analytic_sr dataset 
# Having to ping the Planet V1 API to return the image id's for our required filter
# Filter variables include: Center Coordinate, Date Range, Cloud Cover, Item Type and Asset Type


def build_order(index):
    geojson_geometry = {
    "type": "Point",
    "coordinates": [
        index['properties']['longitude'], index['properties']['latitude']
        ]
    }

    # get images that overlap with our AOI 
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geojson_geometry
    }

    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": start_date,
        "lte": today
      }
    }

    # only get images which have <10% cloud coverage
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.1
      }
    }

    # combine our geo, date, cloud filters
    combined_filter = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }
    
    # API request object
    search_request = {
        "interval": "day",
        "item_types": [item_type],
        "asset_types" : "analytic_sr",
        "filter": combined_filter
    }
    
    search_result = \
      requests.post(
        'https://api.planet.com/data/v1/quick-search',
    #     'https://api.planet.com/data/v2',
        auth=HTTPBasicAuth(PLANET_API_KEY, ''),
        json=search_request)

    return search_result

In [7]:
for index in buffer_wgs84_json_api:
    print(index)
# buffer_wgs84_json_api

{'id': '0', 'type': 'Feature', 'properties': {'Waypoint': 'Orangi River', 'latitude': -2.302313, 'longitude': 34.830777}, 'geometry': {'type': 'Polygon', 'coordinates': [[[34.834211259331184, -2.2988584194452644], [34.834211259331184, -2.305767572007151], [34.827342740668804, -2.305767572007151], [34.827342740668804, -2.2988584194452644], [34.834211259331184, -2.2988584194452644]]]}}


In [8]:
id_list = []

for index in buffer_wgs84_json_api:
    waypoint = index["properties"]["Waypoint"]
    order = build_order(index)
    
    time.sleep(3)
    order = order.json()['features']
    print(len(order))
#     print(order[0])
    


# appending Image ID to `joined_buffer_wgs84_drop_merge` if the analytic_sr is available
# Will only return image id's that meet this requirement.
    for i in order:
        #print(order)
        if "assets.analytic_sr:download" in i["_permissions"]:
            id_list.append((waypoint,i["id"],i["properties"]))

# id_list = list(set(id_list))
id_list

2


[('Orangi River',
  '20200321_075618_92_105c',
  {'acquired': '2020-03-21T07:56:18.927017Z',
   'anomalous_pixels': 0,
   'clear_confidence_percent': 97,
   'clear_percent': 92,
   'cloud_cover': 0.01,
   'cloud_percent': 5,
   'columns': 11333,
   'epsg_code': 32736,
   'ground_control': True,
   'gsd': 4.6,
   'heavy_haze_percent': 0,
   'instrument': 'PS2.SD',
   'item_type': 'PSScene4Band',
   'light_haze_percent': 0,
   'origin_x': 682824,
   'origin_y': 9761664,
   'pixel_resolution': 3,
   'provider': 'planetscope',
   'published': '2020-03-21T20:28:19Z',
   'quality_category': 'test',
   'rows': 8657,
   'satellite_id': '105c',
   'shadow_percent': 3,
   'snow_ice_percent': 0,
   'strip_id': '3249122',
   'sun_azimuth': 84.5,
   'sun_elevation': 62.4,
   'updated': '2020-03-22T04:33:00Z',
   'view_angle': 5,
   'visible_confidence_percent': 85,
   'visible_percent': 95}),
 ('Orangi River',
  '20200320_074142_1010',
  {'acquired': '2020-03-20T07:41:42.751487Z',
   'anomalous_pix

In [9]:
    for i in order:
        print(i["_permissions"])
        print()
        print()

['assets.udm:download', 'assets.analytic:download', 'assets.analytic_xml:download', 'assets.analytic_dn:download', 'assets.analytic_dn_xml:download', 'assets.basic_analytic:download', 'assets.basic_analytic_rpc:download', 'assets.basic_analytic_dn:download', 'assets.basic_analytic_dn_rpc:download', 'assets.basic_analytic_xml:download', 'assets.basic_analytic_dn_xml:download', 'assets.basic_analytic_dn_nitf:download', 'assets.basic_analytic_dn_rpc_nitf:download', 'assets.basic_analytic_dn_xml_nitf:download', 'assets.basic_analytic_nitf:download', 'assets.basic_analytic_rpc_nitf:download', 'assets.basic_analytic_xml_nitf:download', 'assets.basic_udm:download', 'assets.analytic_sr:download', 'assets.basic_udm2:download', 'assets.udm2:download']


['assets.udm:download', 'assets.analytic:download', 'assets.analytic_xml:download', 'assets.analytic_dn:download', 'assets.analytic_dn_xml:download', 'assets.basic_analytic:download', 'assets.basic_analytic_rpc:download', 'assets.basic_analytic_d

In [10]:
# Merging image id's to the dataframe to maintain continuity

image_ids = pd.DataFrame(np.asarray(id_list))
image_ids.rename(columns = {0:'Waypoint', 1:'Image_ID', 2: 'Image_Properties'}, inplace = True) 
image_ids = pd.concat([image_ids.drop(['Image_Properties'], axis=1), pd.json_normalize(image_ids['Image_Properties'])], axis=1)
joined_buffer_wgs84_drop_merge = pd.merge(joined_buffer_wgs84_drop, image_ids, on='Waypoint')
joined_buffer_wgs84_drop_merge

,Waypoint,latitude,longitude,polygon,Image_ID,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,...,satellite_id,shadow_percent,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent
0,Orangi River,-2.302313,34.830777,"POLYGON ((34.83421 -2.29886, 34.83421 -2.30577...",20200321_075618_92_105c,2020-03-21T07:56:18.927017Z,0.00,97,92,0.01,...,105c,3,0,3249122,84.5,62.4,2020-03-22T04:33:00Z,5,85,95
1,Orangi River,-2.302313,34.830777,"POLYGON ((34.83421 -2.29886, 34.83421 -2.30577...",20200320_074142_1010,2020-03-20T07:41:42.751487Z,0.01,99,98,0.01,...,1010,1,0,3246789,86.0,58.6,2020-03-21T04:32:43Z,4,82,99


In [11]:
# Converting list of tuple polygons to list of lists polygons
# This step is necessary to pull the Geometry from `joined_buffer_wgs84_drop_merge`
# and convert to a list of lists...appending to `joined_buffer_wgs84_drop_merge`.

def coord_lister(geom):
    coords = list(geom.exterior.coords)
    return (coords)

coordinates = joined_buffer_wgs84_drop_merge.polygon.apply(coord_lister)
res = []
for poly in coordinates:
    res_2 = list(map(list, poly)) 
    res.append(res_2)

joined_buffer_wgs84_drop_merge['poly_list'] = res
# joined_buffer_wgs84_drop_merge will be used as the basis for all remaining functions
joined_buffer_wgs84_drop_merge

,Waypoint,latitude,longitude,polygon,Image_ID,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,...,shadow_percent,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,poly_list
0,Orangi River,-2.302313,34.830777,"POLYGON ((34.83421 -2.29886, 34.83421 -2.30577...",20200321_075618_92_105c,2020-03-21T07:56:18.927017Z,0.00,97,92,0.01,...,3,0,3249122,84.5,62.4,2020-03-22T04:33:00Z,5,85,95,"[[34.834211259331184, -2.2988584194452644], [3..."
1,Orangi River,-2.302313,34.830777,"POLYGON ((34.83421 -2.29886, 34.83421 -2.30577...",20200320_074142_1010,2020-03-20T07:41:42.751487Z,0.01,99,98,0.01,...,1,0,3246789,86.0,58.6,2020-03-21T04:32:43Z,4,82,99,"[[34.834211259331184, -2.2988584194452644], [3..."


In [12]:
# joined_buffer_wgs84_drop_merge['lat_lon_name'] = f'lat_{joined_buffer_wgs84_drop_merge["latitude"]}_long_{joined_buffer_wgs84_drop_merge["longitude"]}'
joined_buffer_wgs84_drop_merge['lat_lon_name'] = 'lat_'+joined_buffer_wgs84_drop_merge.latitude.map(str)+'_long_'+joined_buffer_wgs84_drop_merge.longitude.map(str)
joined_buffer_wgs84_drop_merge

,Waypoint,latitude,longitude,polygon,Image_ID,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,...,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,poly_list,lat_lon_name
0,Orangi River,-2.302313,34.830777,"POLYGON ((34.83421 -2.29886, 34.83421 -2.30577...",20200321_075618_92_105c,2020-03-21T07:56:18.927017Z,0.00,97,92,0.01,...,0,3249122,84.5,62.4,2020-03-22T04:33:00Z,5,85,95,"[[34.834211259331184, -2.2988584194452644], [3...",lat_-2.302313_long_34.830777
1,Orangi River,-2.302313,34.830777,"POLYGON ((34.83421 -2.29886, 34.83421 -2.30577...",20200320_074142_1010,2020-03-20T07:41:42.751487Z,0.01,99,98,0.01,...,0,3246789,86.0,58.6,2020-03-21T04:32:43Z,4,82,99,"[[34.834211259331184, -2.2988584194452644], [3...",lat_-2.302313_long_34.830777


In [13]:
joined_buffer_wgs84_drop_merge = joined_buffer_wgs84_drop_merge.sort_values(['lat_lon_name', 'updated']) \
    .drop_duplicates('lat_lon_name', keep='last') \
    .sort_index()

joined_buffer_wgs84_drop_merge

,Waypoint,latitude,longitude,polygon,Image_ID,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,...,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,poly_list,lat_lon_name
0,Orangi River,-2.302313,34.830777,"POLYGON ((34.83421 -2.29886, 34.83421 -2.30577...",20200321_075618_92_105c,2020-03-21T07:56:18.927017Z,0.0,97,92,0.01,...,0,3249122,84.5,62.4,2020-03-22T04:33:00Z,5,85,95,"[[34.834211259331184, -2.2988584194452644], [3...",lat_-2.302313_long_34.830777


# List Images

In [14]:
# For Loop 

for index,waypoint_row in joined_buffer_wgs84_drop_merge.iterrows():
    print(waypoint_row.Image_ID)
#     waypoint_row = joined_buffer_wgs84_drop_merge.loc[0]

20200321_075618_92_105c


# Planet API Pull Function

In [15]:

results_paths = pathlib.Path(os.path.join('data', waypoint_row['lat_lon_name'],
                                               f"{waypoint_row['Image_ID']}_3B_AnalyticMS_SR_clip.tif"))
print(results_paths)

data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif


In [16]:
results_paths.exists()

False

In [17]:
# # del results_paths

# for path, something_useless in zip([results_paths, 'something to make this work'):
# #     path = (results_path)
#     print(path.exists())
# #     print(path)
# #     if overwrite or not path.exists():
# #         print(f'downloading {name} to {path}')


In [18]:
# Function goes here.  First step: identify image, polygon, and ro

def planet_api_pull(waypoint_row, overwrite = False):
    
    results_paths = pathlib.Path(os.path.join('data', waypoint_row['lat_lon_name'],
                                               f"{waypoint_row['Image_ID']}_3B_AnalyticMS_SR_clip.tif"))
    if overwrite or not results_paths.exists():


        # Creating the URLs to activate the images...prevents latency during download
        waypoint_row['id0_url'] = f"https://api.planet.com/data/v1/item-types/{item_type}/items/{waypoint_row['Image_ID']}/assets"


        # Returns JSON metadata for assets in this ID. 
        # Learn more: planet.com/docs/reference/data-api/items-assets/#asset
        waypoint_row['activation_link'] = \
              requests.get(
                waypoint_row['id0_url'], #link
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
              )


        # Getting Result Links
        waypoint_row['links'] = waypoint_row['activation_link'].json()[u"analytic_sr"]["_links"]


        # Generating a list of activation links    
        waypoint_row['activation_link'] = waypoint_row['links']["activate"]


        # Request activation of the 'visual' asset:
        # for a in joined_buffer_wgs84_drop_merge['activation_link']:
        activate_result = \
        requests.get(waypoint_row['activation_link'], \
                     auth=HTTPBasicAuth(PLANET_API_KEY, ''))


        # Building the order lists starting with the product information

        waypoint_row['single_product'] = [
                {
                  'item_ids': [waypoint_row['Image_ID']], 
                  'item_type': 'PSScene4Band',
                  'product_bundle': 'analytic_sr'
                }
            ]


        # Setting the clipping boundaries
        waypoint_row['clip'] = [{
            'clip': {
                'aoi': {
                    'type':'Polygon',
                    'coordinates': [waypoint_row['poly_list']] 
                }
            }
        }]


        # create an order request with the clipping tool
        waypoint_row['request_clip'] = {
            'name': 'just clip',
            'products': waypoint_row['single_product'], #single_product,
            'tools': waypoint_row['clip']
        }


        # Creating the url for clipping 
        def place_order(request, auth):
            response = requests.post(orders_v2_url, data=json.dumps(request), auth=auth, headers=headers)

            if response.status_code == 429:
                time.sleep(3)
                response = requests.post(orders_v2_url, data=json.dumps(request), auth=auth, headers=headers)

                if not response.ok:
                    raise Exception(response.content)

            order_id = response.json()['id']

            order_url = orders_v2_url + '/' + order_id

            return order_url


        # order_url
        waypoint_row['order_url'] = place_order(waypoint_row['request_clip'], auth)


        def poll_for_success(order_url, auth, num_loops=100, sleep_time=10):
            count = 0
            print('Waiting for Planet to prep image')

            while(count < num_loops):
                r = requests.get(order_url, auth=auth)
                response = r.json()
                state = response['state']

                print(f'{count * sleep_time}: {state}')
                success_states = ['success', 'partial']

                if state == 'failed':
                    raise Exception(response)
                elif state in success_states:
                    break

                time.sleep(sleep_time)
                count += 1


        # poll_for_success
        poll_for_success(waypoint_row['order_url'],auth)

    #     print('# Downloading from each clip order url')
                                              
        # Establishing S3 client
        s3_resource = boto3.resource('s3')
        s3_client = boto3.client('s3')
                                              
        # Downloading from each clip order url
        def download_order(order_url, auth, overwrite=False):
            time.sleep(20)
            r = requests.get(waypoint_row['order_url'], auth=auth)
            response = r.json()
            results = response['_links']['results']
            results_urls = [r['location'] for r in results if '_3B_AnalyticMS_SR_clip.tif' in r['name']]
            results_names = [r['name'] for r in results if '_3B_AnalyticMS_SR_clip.tif' in r['name']]
            results_paths_old = [pathlib.Path(os.path.join('data', waypoint_row['lat_lon_name'], n)) for n in results_names]
            results_paths = [pathlib.Path(os.path.join('data', waypoint_row['lat_lon_name'], 
                                                        f"{waypoint_row['Image_ID']}_3B_AnalyticMS_SR_clip.tif"))]

                                                                         
            for url, name, path_old, path in zip(results_urls, results_names, results_paths_old, results_paths):
                if overwrite or not path.exists():
                    print(f'downloading {name} to {path}')
                    r = requests.get(url, allow_redirects=True)
                    path.parent.mkdir(parents=True, exist_ok=True)
                    open(path, 'wb').write(r.content)
                else:
                    print(f'{path} already exists, skipping {path}')

            return results_paths


    #     print('# Downloading the orders')

        # Downloading the orders
        waypoint_row['results_path'] = download_order(waypoint_row['order_url'], auth, overwrite)

    else:
        waypoint_row['results_path'] = [results_paths]

#     print('return waypoint_row')
    return waypoint_row


# Get Images

In [19]:
joined_buffer_wgs84_drop_merge.columns

Index(['Waypoint', 'latitude', 'longitude', 'polygon', 'Image_ID', 'acquired',
       'anomalous_pixels', 'clear_confidence_percent', 'clear_percent',
       'cloud_cover', 'cloud_percent', 'columns', 'epsg_code',
       'ground_control', 'gsd', 'heavy_haze_percent', 'instrument',
       'item_type', 'light_haze_percent', 'origin_x', 'origin_y',
       'pixel_resolution', 'provider', 'published', 'quality_category', 'rows',
       'satellite_id', 'shadow_percent', 'snow_ice_percent', 'strip_id',
       'sun_azimuth', 'sun_elevation', 'updated', 'view_angle',
       'visible_confidence_percent', 'visible_percent', 'poly_list',
       'lat_lon_name'],
      dtype='object')

In [20]:
# For Loop 
results_df = pd.DataFrame()
for index,waypoint_row in joined_buffer_wgs84_drop_merge.iterrows():
    print(waypoint_row.Waypoint, waypoint_row.lat_lon_name)
#     waypoint_row = joined_buffer_wgs84_drop_merge.loc[0]

Orangi River lat_-2.302313_long_34.830777


In [22]:
# For Loop 
results_df = pd.DataFrame()
for index,waypoint_row in joined_buffer_wgs84_drop_merge.iterrows():
    print(waypoint_row.Waypoint, waypoint_row.lat_lon_name)
    row = planet_api_pull(waypoint_row, overwrite = False)
    print('planet_api_pull complete!')
    print(row)
    results_df = results_df.append(row)
    print('Next!')
results_df
#     waypoint_row = joined_buffer_wgs84_drop_merge.loc[0]

Orangi River lat_-2.302313_long_34.830777
Waiting for Planet to prep image
0: queued
10: running
20: running
30: running
40: running
50: running
60: running
70: running
80: running
90: running
100: running
110: running
120: success
downloading 319045a4-739e-4317-bf6b-65799c6a3c1b/1/files/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif to data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif
planet_api_pull complete!
Waypoint                                                           Orangi River
latitude                                                               -2.30231
longitude                                                               34.8308
polygon                       POLYGON ((34.83421125933118 -2.298858419445264...
Image_ID                                                20200321_075618_92_105c
acquired                                            2020-03-21T07:56:18.927017Z
anomalous_pixels                                                    

,Image_ID,Waypoint,acquired,activation_link,anomalous_pixels,clear_confidence_percent,clear_percent,clip,cloud_cover,cloud_percent,...,shadow_percent,single_product,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent
0,20200321_075618_92_105c,Orangi River,2020-03-21T07:56:18.927017Z,https://api.planet.com/data/v1/assets/eyJpIjog...,0.0,97.0,92.0,"[{'clip': {'aoi': {'type': 'Polygon', 'coordin...",0.01,5.0,...,3.0,"[{'item_ids': ['20200321_075618_92_105c'], 'it...",0.0,3249122,84.5,62.4,2020-03-22T04:33:00Z,5.0,85.0,95.0


In [ ]:
# # len(orders_downloaded)
# results_df = pd.DataFrame()
# results_df

In [23]:
pathlib.Path(os.path.join('data'))

PosixPath('data')

In [24]:
results_df[['Waypoint', 'latitude', 'longitude', 'results_path']]

,Waypoint,latitude,longitude,results_path
0,Orangi River,-2.302313,34.830777,[data/lat_-2.302313_long_34.830777/20200321_07...


In [25]:
results_df[['Waypoint', 'latitude', 'longitude', 'acquired', 'results_path']].to_csv(pathlib.Path(os.path.join('data/results.csv')))

In [26]:
results_df


,Image_ID,Waypoint,acquired,activation_link,anomalous_pixels,clear_confidence_percent,clear_percent,clip,cloud_cover,cloud_percent,...,shadow_percent,single_product,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent
0,20200321_075618_92_105c,Orangi River,2020-03-21T07:56:18.927017Z,https://api.planet.com/data/v1/assets/eyJpIjog...,0.0,97.0,92.0,"[{'clip': {'aoi': {'type': 'Polygon', 'coordin...",0.01,5.0,...,3.0,"[{'item_ids': ['20200321_075618_92_105c'], 'it...",0.0,3249122,84.5,62.4,2020-03-22T04:33:00Z,5.0,85.0,95.0


In [51]:
test = results_df['results_path'][0]
test

[PosixPath('data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif')]

In [53]:
os.path.relpath(test[0])

'data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif'

In [54]:
for image in results_df['results_path']:
#     infile = image.replace('[PosixPath(\'', '').replace('\')]', '')
    infile = os.path.relpath(image[0])
    print(infile)
    
    if infile[-3:] == "tif":
        outfile = infile[:-3] + "jpg"
        print(outfile)
        gdal.Translate(outfile, infile,options=gdal_Translate_options_string)
        print('\nNext!\n')
        
upload_to_aws('data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif', 
      'w210-planet-data-api', 
      'planet_images/test_baker' + '/'+ 'lat_-2.302313_long_34.830777/conor_is_the_shit_20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif')


data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif
data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.jpg

Next!



In [ ]:
# image_ids = pd.DataFrame(np.asarray(id_list))
# image_ids.rename(columns = {0:'Waypoint', 1:'Image_ID'}, inplace = True) 
# joined_buffer_wgs84_drop_merge = pd.merge(image_ids, joined_buffer_wgs84_drop, on='Waypoint')
# # joined_buffer_wgs84_drop_merge
# # 20200315_080702_17_1057

# Moving files to S3

In [55]:
# Setting rootDir
rootDir = os.path.join(pathlib.Path().absolute(), 'data')
rootDir



'/Users/piper/Desktop/capstone_final_pct_water/notebooks/data'

In [65]:
# Function for moving to S3

import boto3
from botocore.exceptions import NoCredentialsError

ACCESS_KEY = ''
SECRET_KEY = 'CbFCj85+'


def upload_to_aws(local_file, s3_bucket, s3_file):
    s3 = boto3.client('s3')
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)

    try:
        s3. upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [67]:
upload_to_aws('data/lat_-2.302313_long_34.830777/20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif', 
              'w210-planet-data-api', 
              'planet_images/test_baker' + '/'+ 'lat_-2.302313_long_34.830777/conor_is_the_shit_20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif')


Upload Successful


True

In [63]:
# Getting folder structure and calling the 'upload_to_aws' function

for folder, subfolders, filename, *_ in os.walk(rootDir):
#     print(folder)
#     print()
#     print()
#     print(subfolders)
#     print()
#     print()
    print(filename)
#     print()
#     print()
#     print(*_)
#     file = '{}'.format(*filename)
#     folder_s3 = os.path.basename(folder)
#     upload_to_aws(folder+'/'+file, 'w210-planet-data-api', 'planet_images/test_baker' + '/'+ folder_s3 + '/' + file)
# #     print(folder)
# #     print(subfolders)
# #     print(folder_s3)
# #     print(file)

['hymenoptera_data.zip', '.DS_Store', 'results.csv']
[]
['manifest.json']
['20200315_080702_17_1057_metadata.json', '20200315_080702_17_1057_3B_AnalyticMS_DN_udm_clip.tif', '20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif', '20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml']
['20200315_080656_01_1057_3B_AnalyticMS_SR_clip.jpg.aux.xml', '20200321_075618_92_105c_3B_AnalyticMS_SR_clip.jpg.aux.xml', '20200321_075618_92_105c_3B_AnalyticMS_SR_clip.jpg', '20200315_080656_01_1057_3B_AnalyticMS_SR_clip.jpg', '20200315_080658_06_1057_3B_AnalyticMS_SR_clip.tif', '20200321_075618_92_105c_3B_AnalyticMS_SR_clip.tif', '20200320_074142_1010_3B_AnalyticMS_SR_clip.tif', '20200315_080656_01_1057_3B_AnalyticMS_SR_clip.tif']
['20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif', '20200320_064047_100d_3B_AnalyticMS_SR_clip.tif', '20200315_080702_17_1057_3B_AnalyticMS_SR_clip.jpg.aux.xml', '20200315_080702_17_1057_3B_AnalyticMS_SR_clip.jpg']
[]
['manifest.json']
['20200315_080702_17_1057_metadata.jso